In [1]:
import pickle
import os
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import json

In [2]:
with open('./model/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [3]:
def get_review(review):
    sentence_list = []
    sentence_list.append(review)
    sequences = tokenizer.texts_to_sequences(sentence_list)
    input = pad_sequences(sequences, maxlen=200, padding='pre')
    #print(input)
    model = load_model('./model/restaurant.h5')
    output = model.predict(input)
    #print(output[0])
    
    if output[0] >= 0.5:    
        #print("The feedback from user is Positive")    
        return 1
    else:
        #print("The feedback from user is Negative")
        return 0     

In [21]:
get_review("the food was great")

1/1 [==============================] - 2s 2s/step


1

In [4]:
app = Flask(__name__)
CORS(app)

@app.route('/',methods=['GET'])
def Home():
    return render_template('index.html')

@app.route('/review', methods = ['POST'])
def review():
    if request.method == 'POST':
        record = request.get_json()    
        user_feedback = record['review']
        prediction_val = get_review(user_feedback)    
        prediction = {}
        prediction['sentiment'] = prediction_val
        prediction['review'] = record['review']
        print(prediction)        
        return json.dumps(prediction)
    
if __name__ == "__main__":
    app.run(use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Dec/2022 18:09:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2022 18:09:44] "GET /static/images/like.png HTTP/1.1" 304 -
127.0.0.1 - - [14/Dec/2022 18:09:44] "GET /static/images/dislike.png HTTP/1.1" 304 -
127.0.0.1 - - [14/Dec/2022 18:09:44] "GET /static/images/brand_logo.png HTTP/1.1" 304 -
127.0.0.1 - - [14/Dec/2022 18:09:44] "GET /static/images/restaurant_image.png HTTP/1.1" 304 -


1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [14/Dec/2022 18:10:56] "POST /review HTTP/1.1" 200 -


{'sentiment': 0, 'review': "Food was pretty good, I had the loaded burger with sweet potatoe fries, can't really go wrong with that and it was tasty. My daughter had the smoked salmon with avocado bagel and did not enjoy it."}
1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [14/Dec/2022 18:12:11] "POST /review HTTP/1.1" 200 -


{'sentiment': 1, 'review': 'Everything is perfect. Staff, manager, food, ambiance….delicious courteous cozy… I love this place…..Thank you Emily for your services and suggestions for beer'}
